In [ ]:
# Tool importing section
import os #related to file directory
import earthpy as et # not in use
import earthpy.plot as ep # plotting histogram
import earthpy.spatial as es # used for cropping raster
from glob import glob #identifying files within path
import geopandas as gpd #reading files - vector
import numpy as np #used for masking rasters
import numpy.ma as ma #used for masking rasters
import matplotlib.pyplot as plt #plotting data
import matplotlib.lines as mlines #dissolve
from osgeo import gdal,ogr #used to convert raster to shp
import pathlib #related to file directory
from pathlib import Path #related to file directory
import rasterio as rio #reading files - raster
from rasterio.merge import merge #mosaic rasters
from rasterio.plot import show #plotting mosaic raster
from rasterio.features import shapes #create vector from raster
from rasterio.warp import calculate_default_transform, reproject, Resampling #projecting rasters
from rasterio.plot import plotting_extent #set extent after cropping
import richdem as rd #open raster for slope calculations
from shapely.geometry import Polygon #clip vector
from shapely.geometry import shape #vector from raster process
import sys #gdal error message

In [ ]:
#Set Highest level folder
os.chdir(os.path.join('P:\\Personal Files\\Education\\FRCC\\NSF_Internship\\NSF_Project_Files\\Data'))
#Set variables
projection = "epsg:26914"
dem_mosaic_outpath = os.path.join("Slope_Results","dem_mosaic.tif")
dem_mosaic_crop_outpath = os.path.join("Slope_Results","dem_mosaic_crop.tif")
slope_outpath = os.path.join("Slope_Results","slope.tif")
slope_rc_outpath = os.path.join("Slope_Results","slope_reclass.tif")
poly_slope = os.path.join("Slope_Results","slope_reclass_poly.shp")
slope_risk = os.path.join("Final_Results","Input_Layers","Slope_Risk.shp")
slope_fig = os.path.join("Figures","Reservation_Slope_by_Risk_Value.png")

In [ ]:
#Set Reservation boundary
res_field = "LARName" #field name where reservation name is found
res_name = "Standing Rock LAR" #name of the reservation from the shape file
res_path = os.path.join("Reservation_Boundary_Layer","BIA_National_LAR.shp")
res_boundary = gpd.read_file(res_path)
res_aoi = res_boundary[res_boundary[res_field] == res_name]

In [ ]:
#Access Dems
dem_path = os.path.join("Slope_Original_Data")
all_dems = glob(os.path.join(dem_path, "*.tif"))
all_dems.sort()
#Set Reprojected Dem folder
reproj_dem_fold = os.path.join("Slope_Results","Slope_Projection")

In [ ]:
#Start of the applying Projection Area

In [ ]:
#Change Projections to match
def reproject_et(inpath, outpath, new_crs):
    dst_crs = new_crs #new projection

    with rio.open(inpath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rio.open(outpath, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [ ]:
for ned in all_dems:
    reproject_et(inpath = os.path.join(ned), 
                 outpath = os.path.join(reproj_dem_fold,os.path.basename(ned)), 
                 new_crs = projection)

In [ ]:
projected_dem_path = glob(os.path.join(reproj_dem_fold,"*.tif"))

In [ ]:
#End of the applying Projection Area

In [ ]:
#Start of Merging (Mosaic) and Masking Rasters

In [ ]:
dems_to_mosaic = []
#Open Rasters
for ned in projected_dem_path:
    src = rio.open(ned)
    dems_to_mosaic.append(src)

In [ ]:
dem_mosaic, dem_out_trans = merge(dems_to_mosaic)

In [ ]:
with rio.open(projected_dem_path[0]) as src:
    dem_data = src.read()
    dem_meta = src.profile
    
dem_meta

In [ ]:
dem_width_meta = dem_mosaic.shape[2]
dem_height_meta = dem_mosaic.shape[1]

In [ ]:
dem_meta['width'] = dem_width_meta
dem_meta['height'] = dem_height_meta
dem_meta['transform'] = dem_out_trans
dem_meta

In [ ]:
for raster in dems_to_mosaic:
    raster.close()

In [ ]:
mask_dem_mosaic = np.where(dem_mosaic < 0, True, False)
masked_dem_mosaic = np.ma.masked_array(dem_mosaic, mask_dem_mosaic)
#ep.hist(masked)

In [ ]:
#End of Merging (Mosaic) and Masking Rasters

In [ ]:
#Converting Mosaic Numpy to a GeoTiff and Cropping

In [ ]:
sqz_dem_mosaic = masked_dem_mosaic.squeeze()

In [ ]:
# Write raster object to folder
with rio.open(dem_mosaic_outpath, 'w', **dem_meta) as dst:
    dst.write(sqz_dem_mosaic, 1)

In [ ]:
#clearing out nolonger needed data.
del dem_mosaic
del dem_out_trans
del projected_dem_path
del mask_dem_mosaic
del masked_dem_mosaic
del sqz_dem_mosaic

In [ ]:
with rio.open(dem_mosaic_outpath) as dem_src:

    # Project boundary to match raster data
    res_projected = res_aoi.to_crs(dem_src.crs)

    # Crop raster data to boundary
    dem_data_crop, dem_meta_crop = es.crop_image(
        dem_src, res_projected)

# Define plotting extent using cropped array and transform from metadata
dem_crop_plot_extent = plotting_extent(
    dem_data_crop[0], dem_meta_crop["transform"])

In [ ]:
f, ax = plt.subplots()

ep.plot_bands(dem_data_crop,
              ax=ax,
              title="Check that cropped DEMs line up with Reservation Shapefile",
              scale=False,
              cmap="gray",
              extent=dem_crop_plot_extent)  # Use plotting extent from cropped array

res_projected.plot(color='None',
                    edgecolor='teal',
                    linewidth=2,
                        ax=ax)

plt.show()

In [ ]:
sqz_dem_data_crop = dem_data_crop.squeeze()
with rio.open(dem_mosaic_crop_outpath, 'w', **dem_meta_crop) as dst:
    dst.write(sqz_dem_data_crop, 1)

In [ ]:
#End of Converting Mosaic Numpy to a GeoTiff and Cropping

In [ ]:
#clearing out old data round 2
del dem_mosaic_outpath
del dem_data_crop
del sqz_dem_data_crop

In [ ]:
#Create GRID Slope

In [ ]:
res_dem = rd.LoadGDAL(dem_mosaic_crop_outpath)
#plt.imshow(res_dem, interpolation='none')
#plt.colorbar()
#plt.show()

In [ ]:
slope = rd.TerrainAttribute(res_dem, attrib='slope_percentage')
#rd.rdShow(slope, axes=False, cmap='magma', figsize=(8, 5.5))
#plt.show()

In [ ]:
#End of Create GRID Slope

In [ ]:
#Reclassify GRID Slop

In [ ]:
print('Slope % Min: ',slope.min())
print('Slope % Max: ',slope.max())
print(slope)
slope_reclass = slope
slope_reclass[(slope_reclass >=1)&(slope_reclass <=5)]=2
slope_reclass[slope_reclass >5]=1
slope_reclass[slope_reclass <1]=6
print(slope_reclass)
print('Slope Reclass Min: ',slope_reclass.min())
print('Slope Reclass Max: ',slope_reclass.max())

In [ ]:
slope_org = rd.TerrainAttribute(res_dem, attrib='slope_percentage')
#f, ax1 = plt.subplots(figsize=(8, 5.5))
#ep.plot_bands(slope_reclass, ax=ax1,title="slope reclassified",cmap='terrain', figsize=(8, 5.5))
#rd.rdShow(slope_org, axes=False, cmap='terrain', figsize=(8, 5.5))
#plt.show()

In [ ]:
with rio.open(slope_outpath, 'w', **dem_meta_crop) as dst:
    dst.write(slope_org, 1)
with rio.open(slope_rc_outpath, 'w', **dem_meta_crop) as dst:
    dst.write(slope_reclass, 1)

In [ ]:
#End of Reclassify GRID Slop

In [ ]:
#clearing out old data round 3
del dem_mosaic_crop_outpath
del res_dem
del slope
del slope_reclass
del slope_org

In [ ]:
#Convert Reclassified Grid into vector

In [ ]:
mask = None
with rio.Env():
    with rio.open(slope_rc_outpath) as src:
        image = src.read(1) # first band
        #print(src.crs)
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

In [ ]:
geoms = list(results)
#print (geoms)

In [ ]:
#print (shape(geoms[0]['geometry']))

In [ ]:
gpd_polygonized_raster = gpd.GeoDataFrame.from_features(geoms,crs=projection)
gpd_polygonized_raster.to_file(poly_slope)

In [ ]:
#End of Convert Reclassified Grid into vector

In [ ]:
#clearing out old data round 4

In [ ]:
del slope_rc_outpath
del geoms
del gpd_polygonized_raster

In [ ]:
#Dissolve boundaries based on Slope_Risk and save

In [ ]:
slope_poly = gpd.read_file(poly_slope)
slope_value = slope_poly[['raster_val','geometry']]
slope_dissolve = slope_value.dissolve(by='raster_val')
slope_dissolve

In [ ]:
slope_clip = gpd.clip(slope_dissolve,res_projected)

In [ ]:
f, ax1 = plt.subplots()
slope_clip.plot(ax=ax1)
ax1.set(title="Slope Reclassified")
plt.show()
plt.draw()
f.savefig(slope_fig, dpi=400)

In [ ]:
#Save File
slope_clip.to_file(slope_risk)

In [ ]:
#End of DEM Prep